En este cuaderno se explica a detalle el procedimiento seguido para ejecutar el algoritmo que encuentra aproximaciones a los distintos frentes de Pareto.

In [4]:
import matplotlib.pyplot as plt
from utils.func_aux import *
from utils.func_vis import *
import pandas as pd


# Opciones de matplotlib
rc = plt.rcParams
rc["figure.figsize"] = [15, 5]

# Para mostrar todas las columnas cuando se imprime un df
pd.set_option("display.max_columns", None)

# Para poner el estilo de las gráficas de matplotlib parecido al de ggplot
plt.style.use("ggplot") 

go_to_PCUI_Proyect()

Primero se encuentra una forma de correr el algoritmo de PFI-EMOA, pero cambiando la escalarización de la combinación de Tchebychev. 

Necesitamos para correr el programa: 
- El problema a resolver (en nuestro caso: DTLZ1-9, WFG1-9, aunque se pueden escoger otros)
- El número de objetivos (en nuestro caso: 2,3,4,5,6,7 para todos los problemas)
- El número de variables de decisión  (cada problema requiere un número de variables de decisión distintos para cada número de objetivos)
- El valor específico de cada coeficiente de la escalarización 

En este caso la escalarización se realiza con la función aumentada de Tchebychev: 

$$ \text{ATCH}_{\vec{w}}(\vec{x}) = \max_{i=1,\ldots,k} \{x_iw_i\} + \alpha \sum_{i=1}^k x_i $$
con $\alpha>0$ y $\vec{w}\in {\mathbb{R}^+}^k$ que satisface $\sum_i w_i =1$

Donde para nuestros indicadores tenemos 

$$\text{ATCH}_{\vec{w}}(\vec{I}_s(\mathcal{A},\mathcal{Z})=\max\left\{ w_0 \text{IGD}^+(\mathcal{A},\mathcal{Z}), w_1 E_s(\mathcal{A})\right\} +\alpha\left(w_0 \text{IGD}^+(\mathcal{A},\mathcal{Z})+ w_1 E_s(\mathcal{A})\right) $$


es decir el primer peso $w_0$ es el que caracteriza la importancia que le damos al indicador $\text{IGD}^+$

# Generación de archivos .pof y .pof.nd con PCUI


## Definiendo los $w_0$

Se crea una función que regresa el intervalo 0,1 dividido en el número que indiquemos. El primero y el último se ponen con valores muy cercanos a 0 y 1 porque al correr los resultados se obtenían valores idénticos en cada uno

In [2]:
ws = get_w_espaciados(10)
ws

[(0.001, 0.999),
 (0.1, 0.9),
 (0.2, 0.8),
 (0.3, 0.7),
 (0.4, 0.6),
 (0.5, 0.5),
 (0.6, 0.4),
 (0.7, 0.3),
 (0.8, 0.2),
 (0.9, 0.1),
 (0.999, 0.001)]

## Creando archivo de configuración

Los archivos de configuración tienen toda la información sobre las características del algoritmo a correr:
- Combinación de pesos específica en la esclarización de Tchebychev
- El número de evaluaciones de la función de aptitud
- el número de variables de decisión
- El número de variables objetivo
- Una etiqueta para diferenciar a cada archivo


In [5]:
filename = create_param_file(w_comb=ws[5],label=5,feval=50_000,n_obj=2,n_var=12,wfg_npos=10)
path=f'./demo/input/{filename}'
print(path)

./demo/input/Param_02D-5.cfg


'Fri Sep 29 17:00:56 2023'

## Ejecutando algoritmo

Para correr el programa primero tenemos que tener todos los parámetros de cada uno de los problemas. 

Para esto se obtuvieron el número de variables de decisión de acuerdo al número de objetivos de acuerdo a `benchmark.c` para DTLZ y ZDT

### ZDT
Estos problemas todos tienen 30 excepto ZDT4 y ZDT6 que tienen 10. No se considera ZDT5 por tener variables binarias

### DTLZ 

$$n^{(\text{DTLZ})}_{var}=n^{(\text{DTLZ})}_{obj}+k-1$$
con $k=5$ para DTLZ1, $k=10$ para DTLZ2-DTLZ6 y $k=20$ para DTLZ7 .


### WFGF
Para WFG se tomó la convención del artículo
$$n^{(\text{WFG})}_{var}=24+2(n^{(\text{WFG})}_{obj}-2),\\ n^{(\text{WFG})}_{pos}=2(n^{(\text{WFG})}_{obj}-1)$$

In [8]:
# problemas = {
#     "ZDT1": {"n_pos":[],"n_var": [], "n_obj": [2] },
#     "ZDT2": {"n_pos":[],"n_var": [], "n_obj": [2] },
#     "ZDT3": {"n_pos":[],"n_var": [], "n_obj": [2] },
#     "ZDT4": {"n_pos":[],"n_var": [], "n_obj": [2] },
#     "ZDT6": {"n_pos":[],"n_var": [], "n_obj": [2] },
#     "DTLZ1": {"n_pos":[],"n_var": [], "n_obj": [2,3,4,5,6,7,10] },
#     "DTLZ2": {"n_pos":[],"n_var": [], "n_obj": [2,3,4,5,6,7,10] },
#     "DTLZ3": {"n_pos":[],"n_var": [], "n_obj": [2,3,4,5,6,7,10] },
#     "DTLZ4": {"n_pos":[],"n_var": [], "n_obj": [2,3,4,5,6,7,10] },
#     "DTLZ5": {"n_pos":[],"n_var": [], "n_obj": [2,3,4,5,6,7,10] },
#     "DTLZ6": {"n_pos":[],"n_var": [], "n_obj": [2,3,4,5,6,7,10] },
#     "DTLZ7": {"n_pos":[],"n_var": [], "n_obj": [2,3,4,5,6,7,10] },
#     "WFG1": {"n_pos":[],"n_var": [], "n_obj": [2,3,4,5,6,7,10] },
#     "WFG2": {"n_pos":[],"n_var": [], "n_obj": [2,3,4,5,6,7,10] },
#     "WFG3": {"n_pos":[],"n_var": [], "n_obj": [2,3,4,5,6,7,10] },
#     "WFG4": {"n_pos":[],"n_var": [], "n_obj": [2,3,4,5,6,7,10] },
#     "WFG5": {"n_pos":[],"n_var": [], "n_obj": [2,3,4,5,6,7,10] },
#     "WFG6": {"n_pos":[],"n_var": [], "n_obj": [2,3,4,5,6,7,10] },
#     "WFG7": {"n_pos":[],"n_var": [], "n_obj": [2,3,4,5,6,7,10] },
#     "WFG8": {"n_pos":[],"n_var": [], "n_obj": [2,3,4,5,6,7,10] },
#     "WFG9": {"n_pos":[],"n_var": [], "n_obj": [2,3,4,5,6,7,10] },
# }

# problema_lista,n_var_lista,n_obj_lista,n_pos_lista=[],[],[],[]

# for k,v in problemas.items():
#     for obj in v['n_obj']:
#         n_obj_lista.append(obj)
#         if k.startswith('DTLZ'):
#             num_DTLZ=int(k[-1])
#             if num_DTLZ in [2,3,4,5,6]:
#                 k_const=10
#             elif num_DTLZ==1:
#                 k_const=5
#             else:
#                 k_const=20
#             problema_lista.append(k)
#             n_var_lista.append(obj+k_const-1)
#             n_pos_lista.append(4)
#             v['n_var'].append(obj+k_const-1)
#             v['n_pos'].append(4)

#         if k.startswith('WFG'):
#             v['n_var'].append(24+2*(obj-2))
            
#             v['n_pos'].append(2*(obj-1))
#             problema_lista.append(k)
#             n_var_lista.append(24+2*(obj-2))
#             n_pos_lista.append(2*(obj-1))

#         if k.startswith('ZDT'):
#             num_ZDT=int(k[-1])

#             problema_lista.append(k)
#             n_pos_lista.append(4)
#             if num_ZDT==4 or num_ZDT==6:
#                 n_var_lista.append(10)
#                 v["n_var"].append(10)
#             else:
#                 n_var_lista.append(30)
#                 v['n_var'].append(30)
#             v['n_pos'].append(4)
               

In [6]:
   
go_to_Assesment()
pd.set_option('display.max_rows', None)
# df_var_obj=pd.DataFrame({'problemas':problema_lista,'n_obj':n_obj_lista,'n_var':n_var_lista,'n_pos':n_pos_lista}).sort_values(['problemas','n_obj'])
# df_var_obj.to_csv('../tablas_generadas/problemas_prueba.csv',index=False)
# problemas

df_var_obj=pd.read_csv('../tablas_generadas/problemas_prueba.csv')
df_var_obj

,problemas,n_obj,n_var,n_pos
0,DTLZ1,2,6,4
1,DTLZ1,3,7,4
2,DTLZ1,4,8,4
3,DTLZ1,5,9,4
4,DTLZ1,6,10,4
5,DTLZ1,7,11,4
6,DTLZ1,10,14,4
7,DTLZ2,2,11,4
8,DTLZ2,3,12,4
9,DTLZ2,4,13,4


Así, podemos crear el archivo de configuración para cada uno de los problemas de interés en las dimensiones que nos interesen.

Podemos especificar la elección de indicadores a combinar en Tchebycheff

# Generando pesos para R2

## Generando pesos con SLD

Finalmente  se prefirió usar el método de [S-energy](#generando-pesos-usando-s-energy) para generar los pesos de las funciones de utilidad porque con esta función, los vectores aparecían en diferentes cardinalidades.

In [8]:
import numpy as np
import numpy.matlib
import itertools
import math

def SLD(N, dim):
    """Generates approximately N uniformly distributed points using the Simplex-Lattice Design with two layers"""
    H1 = 1
    while (math.comb(H1+dim, dim-1) <= N):
        H1 += 1
    W = nchoosek(list(range(1, H1+dim)), dim-1)-np.matlib.repmat(np.arange(0, dim-1), math.comb(H1+dim-1, dim-1), 1)-1
    W = (np.hstack((W, np.zeros([len(W), 1])+H1))-np.hstack((np.zeros([len(W), 1]), W)))/H1
    if (H1 < dim):
        H2 = 0
        while (math.comb(H1+dim-1, dim-1)+math.comb(H2+dim, dim-1) <= N):
            H2 += 1
        if (H2 > 0):
            W2 = nchoosek(list(range(1, H2+dim)), dim-1)-np.matlib.repmat(np.arange(0, dim-1), math.comb(H2+dim-1, dim-1), 1)-1
            W2 = (np.hstack((W2, np.zeros([len(W2), 1])+H2))-np.hstack((np.zeros([len(W2), 1]), W2)))/H2
            W2 = W2/2+1/(2*dim)
            W = np.vstack((W, W2))
    N = len(W)
    go_to_PCUI_Proyect()
    np.savetxt(f'./SLD_{str(dim).zfill(2)}D_{N}.sld', W, '%.6f', header=str(N)+' '+str(dim))
    return

def nchoosek(v, k):
    """Returns a matrix with all combinations of v taken k at a time"""
    return np.array(list(itertools.combinations(v, k)))

# SLD(120,7)

## Generando pesos usando S-energy

Con [pymoo](https://pymoo.org/misc/reference_directions.html) podemos obtener las direcciones de referencia en casos más generales. En este caso lo hacemos con el método de S-energy

In [4]:
from pymoo.util.ref_dirs import get_reference_directions
from pymoo.visualization.scatter import Scatter


def write_file(path,array,n_objetivoses,puntos=120):
    if os.path.exists(path):
        os.remove(path)
    
    s=f'# {puntos} {n_objetivoses}\n'
    for ai in array:
        for aii in ai:
            s+=f'{round(aii,6)} '
        s.strip()
        s+='\n'
    
    with open(file=path,mode='w') as f:
        f.write(s)
    
    return s

puntos=120
# for n_objetivos in [2,3,4,5,6,7]:
#     ref_dirs = get_reference_directions("energy", n_objetivos, 120, seed=1)
#     write_file(path=f'./demo/input/weight/weight_0{n_objetivos}D_{puntos}.senergy',array=ref_dirs,n_objetivoses=n_objetivos)

# Generando archivo de configuración

Hasta el final se especifica la ruta de las direcciones de referencia que generamos previamente

In [5]:

def create_param_file(w_comb, label, feval=50000, n_obj=3, n_var=26, wfg_npos=4):
    """Crea archivo de configuración para ejecutar el algoritmo especificando
    w_comb: combinación de pesos para la escalarización
    label: nombre del archivo
    feval: número de evaluaciones de la función de aptitud
    n_obj: variables objetivo
    n_var: variables de decisión
    wfg_npos: parámetros de posición para los problemas de WFG

    Regresa el nombre del arhivo para encontrarlo con los siguientes programas
    """

    go_to_PCUI_Proyect()

    # Se lee y modifica un template
    src = r"./demo/input/Param_03D_template.cfg"
    dst = rf"./demo/input/Param_{str(n_obj).zfill(2)}D-{label}.cfg"

    # Eliminar si ya existe
    if os.path.exists(dst):
        os.remove(dst)

    # Leer el template y construir el contenido del nuevo archivo
    with open(src, "r") as f:
        s = f.read()
        s = re.sub(
            r"feval = \d+", f"feval = {feval}", s
        )  # cambiar el número de evaluaciones
        s = re.sub(
            r"nobj = \d+", f"nobj = {n_obj}", s
        )  # cambiar el número de objetivos
        s = re.sub(
            r"^nvar = \d+", f"nvar = {n_var}", s, flags=re.MULTILINE
        )  # cambiar el número de variables
        s = re.sub(
            r"^wfg_npos = \d+", f"wfg_npos = {wfg_npos}", s, flags=re.MULTILINE
        )  # cambiar el numero de pos related parameters para los wfg
        s=re.sub(r"^wfile_r2 = input/weight/weight_03D_14.sld",f"wfile_r2 = input/weight/weight_{str(n_obj).zfill(2)}D_120.senergy",s,flags=re.MULTILINE)

    # Crear y escribir el nuevo archivo
    s += f"\n\nw_comb = {w_comb[0]}, {w_comb[1]}\np_Deltap = 2"

    with open(dst, "w") as f:
        f.write(s)

    return dst[dst.find("Param") :]

create_param_file(w_comb=ws[1],label=1,feval=50000,n_obj=5,n_var=26)


'Param_05D-1.cfg'

In [6]:
def ejecutar_algoritmo_con_input(
    w_comb, label,n_obj,n_var, problema_prueba, ind_conv='IGD+', ind_diversity='ES',algoritmo='pcuiemoa', feval=50_000,num_ejecuciones=10, func_escalarizacion='augmented_chebyshev_pcui',wfg_npos=4
):
    """Se ejecuta el algoritmo de acuerdo a los parámetros indicados, se guarda en output y se le cambia el nombre para que tenga el w_0 usado"""
    go_to_PCUI_Proyect()

    param_input=create_param_file(w_comb=w_comb,label=label,n_obj=n_obj,n_var=n_var,feval=feval,wfg_npos=wfg_npos)

    os.chdir("./demo")

    if algoritmo == "pcuiemoa":
        s = f"./{algoritmo} input/{param_input} {problema_prueba} {num_ejecuciones} {func_escalarizacion} {ind_conv} {ind_diversity}"
        params = re.findall(r"_(.+)\-", param_input)[0]
        path_out_lista_pof = [
            f"./demo/output/PCUI-EMOA_ATCH_{ind_conv}_{ind_diversity}_{problema_prueba}_{params}_R{str(i+1).zfill(2)}.pof"
            for i in range(num_ejecuciones)
        ]
        path_out_lista_pos = [
            f"./demo/output/PCUI-EMOA_ATCH_{ind_conv}_{ind_diversity}_{problema_prueba}_{params}_R{str(i+1).zfill(2)}.pos"
            for i in range(num_ejecuciones)
        ]
    else:
        s = f"./emo_moea {algoritmo} input/{param_input} {problema_prueba} {num_ejecuciones}"

    print(f"Ejecutando comando\n{s}")
    print("-" * 100)
    os.system(s)
    print("-" * 100)
    print("Comando terminado")

    # Renombrar archivo
    go_to_PCUI_Proyect()

    for i in range(len(path_out_lista_pof)):
        file_pof=path_out_lista_pof[i]
        file_pos=path_out_lista_pos[i]
        idx_d=file_pof.find('D_')

        try:
            os.rename(file_pof,f'{file_pof[:idx_d+2]}{str(label).zfill(2)}W_{file_pof[idx_d+2:]}')
        except:
            print('Error de copia en ',file_pof)
        try:
            os.rename(file_pos,f'{file_pos[:idx_d+2]}{str(label).zfill(2)}W_{file_pos[idx_d+2:]}')
        except:
            print('Error de copia en ',file_pos)
    

    return path_out_lista_pof

## Ejecuciones para todos los problemas

In [11]:
rango=df_var_obj.query('n_obj<10').iloc[42:-5]
pd.set_option('display.max_rows', None)
print(len(rango))
rango

54


,problemas,n_obj,n_var,n_pos
49,WFG1,2,24,2
50,WFG1,3,26,4
51,WFG1,4,28,6
52,WFG1,5,30,8
53,WFG1,6,32,10
54,WFG1,7,34,12
56,WFG2,2,24,2
57,WFG2,3,26,4
58,WFG2,4,28,6
59,WFG2,5,30,8


In [12]:
for j, row in rango.iterrows():
    prob,n_obj,n_var,wfg_npos=row
    
    for i,wii in enumerate(ws):
        print(prob,n_obj,n_var,wfg_npos,wii)

        ejecutar_algoritmo_con_input(w_comb=wii,label=i,n_obj=n_obj,n_var=n_var,problema_prueba=prob,wfg_npos=wfg_npos,ind_conv='R2')

# Tarda aproximadamente 6 min cada ejecución por cada peso

WFG1 2 24 2 (0.001, 0.999)
Ejecutando comando
./pcuiemoa input/Param_02D-0.cfg WFG1 10 augmented_chebyshev_pcui R2 ES
----------------------------------------------------------------------------------------------------
log tmp PCUI-EMOA_ATCH_R2_ES_WFG1_0.log
PCUI-EMOA
Combination utility function:
Utility: augmented_chebyshev_pcui
Using w_comb = (0.001000, 0.999000)
R2 utility function:
Utility: achievement_scalarizing_function
Random inicialization of population.
mu 120, var 26, obj 2
Codificacion real
caso 1 = 8511   caso 2 = 3409
Random inicialization of population.
mu 120, var 26, obj 2
Codificacion real
caso 1 = 8149   caso 2 = 5550
Random inicialization of population.
mu 120, var 26, obj 2
Codificacion real
caso 1 = 8188   caso 2 = 5155
Random inicialization of population.
mu 120, var 26, obj 2
Codificacion real
caso 1 = 8487   caso 2 = 5115
Random inicialization of population.
mu 120, var 26, obj 2
Codificacion real
caso 1 = 7726   caso 2 = 6867
Random inicialization of populati

# Moviendo los archivos a la carpeta `archivos_w`

Los movemos a `../archivos_w/w_0i` para cada una de las combinaciones de $w_0$

In [5]:
go_to_PCUI_Proyect()

for j, row in df_var_obj.iterrows():
    prob,n_obj,n_var,wfg_npos=row
    
    for i,wii in enumerate(ws):
        # print(prob,n_obj,n_var,wfg_npos,wii[0])
        wi=f'w_0{i}'
        for r in range(1,11):

            run_str=str(r).zfill(2)

            if n_obj>9:
                continue
            
            n_obj_str=str(n_obj).zfill(2)

            path_pof=f'./demo/output/PCUI-EMOA_ATCH_R2_ES_{prob}_{n_obj_str}D_{str(i).zfill(2)}W_R{run_str}.pof'
            path_pos=f'./demo/output/PCUI-EMOA_ATCH_R2_ES_{prob}_{n_obj_str}D_{str(i).zfill(2)}W_R{run_str}.pos'
            
            if not os.path.exists(path_pof) or not os.path.exists(path_pos):
                print('No existen: ',path_pos,path_pof)
            else:
                print('Archivo copiado: ',shutil.copy(src=path_pof,dst=f'../../archivos_w_R2/{wi}/PCUIEMOA_{prob}_{n_obj_str}D_R{run_str}.pof'))
                print('Archivo copiado: ',shutil.copy(src=path_pos,dst=f'../../archivos_w_R2/{wi}/PCUIEMOA_{prob}_{n_obj_str}D_R{run_str}.pos'))

     


Archivo copiado:  ../../archivos_w_R2/w_00/PCUIEMOA_DTLZ1_02D_R01.pof
Archivo copiado:  ../../archivos_w_R2/w_00/PCUIEMOA_DTLZ1_02D_R01.pos
Archivo copiado:  ../../archivos_w_R2/w_00/PCUIEMOA_DTLZ1_02D_R02.pof
Archivo copiado:  ../../archivos_w_R2/w_00/PCUIEMOA_DTLZ1_02D_R02.pos
Archivo copiado:  ../../archivos_w_R2/w_00/PCUIEMOA_DTLZ1_02D_R03.pof
Archivo copiado:  ../../archivos_w_R2/w_00/PCUIEMOA_DTLZ1_02D_R03.pos
Archivo copiado:  ../../archivos_w_R2/w_00/PCUIEMOA_DTLZ1_02D_R04.pof
Archivo copiado:  ../../archivos_w_R2/w_00/PCUIEMOA_DTLZ1_02D_R04.pos
Archivo copiado:  ../../archivos_w_R2/w_00/PCUIEMOA_DTLZ1_02D_R05.pof
Archivo copiado:  ../../archivos_w_R2/w_00/PCUIEMOA_DTLZ1_02D_R05.pos
Archivo copiado:  ../../archivos_w_R2/w_00/PCUIEMOA_DTLZ1_02D_R06.pof
Archivo copiado:  ../../archivos_w_R2/w_00/PCUIEMOA_DTLZ1_02D_R06.pos
Archivo copiado:  ../../archivos_w_R2/w_00/PCUIEMOA_DTLZ1_02D_R07.pof
Archivo copiado:  ../../archivos_w_R2/w_00/PCUIEMOA_DTLZ1_02D_R07.pos
Archivo copiado:  ..

## Obteniendo soluciones no dominadas

También se tiene una manera de quedarse sólo con las soluciones no dominadas. 


se ejecuta la indicación `./emo_ndset output/NSGA2_ZDT1_02D_R01.pof` para cada archivo y se obtiene en el mismo directorio un 
¿Es necesario hacerlo para calcular indicadores?

In [29]:
path='../../archivos_w/w_00/PCUIEMOA_DTLZ1_03D_R01.pof'
# os.system(f'./demo/emo_ndset {path}')

0

In [33]:
# w_path=[f'../../archivos_w/w_0{i}' for i in range(11)]

# for wii in w_path:
#     for file in os.listdir(wii):
#         if file.endswith('.pof'):
#             os.system(f'./demo/emo_ndset {wii}/{file}')

Obtenemos soluciones no dominadas para todos los problemas y las pasamos a una estructura de folders de modo que sólo tenemos los output y los que están en `archivos_w/w_i` en el root. 

Guardé todos los archivos .pof como df así que por ahí podría venir el problema. La columna extra trae el valor de w_0 así que seguramente es un error de pasar los archivos de un lado a otro